In [ ]:
# Download a file based on its file ID.
file_id = '18DbC6Xj4NP-hLzI14WuMaAEyq482vNfn'
# Download dataset
!gdown https://drive.google.com/uc?id={file_id}
# Unzip the downloaded file
!unzip -q PlantVillage.zip

Downloading...
From: https://drive.google.com/uc?id=18DbC6Xj4NP-hLzI14WuMaAEyq482vNfn
To: /content/PlantVillage.zip
100% 866M/866M [00:04<00:00, 193MB/s]


import libraries

In [ ]:
!pip install optimizers

     |████████████████████████████████| 63 kB 1.1 MB/s 
  Created wheel for optimizers: filename=Optimizers-2.1-py3-none-any.whl size=2284 sha256=c70ff239259aeb33d58fa376bb2cb4fe79ded361c8e79440e756d55f5939be61
  Stored in directory: /root/.cache/pip/wheels/56/a5/4d/f679a391b5fca0b18c5e2fcd66ebff8900d97d6d95713915b9
Successfully built optimizers
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import keras
import numpy as np
import pickle
import cv2
import os
import matplotlib.pyplot as plt
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [ ]:
# Dimension of resized image 
DEFAULT_IMAGE_SIZE = tuple((256, 256))
# Number of images used to train the model
N_IMAGES = 100
# Path to the dataset folder 
root_dir = './PlantVillage'
train_dir = os. path.join(root_dir, 'train') 
val_dir = os.path.join(root_dir, 'val')

In [ ]:
def convert_image_to_array(image_dir):
  try:
    image = cv2.imread (image_dir)
    if image is not None:
      image = cv2.resize(image, DEFAULT_IMAGE_SIZE)
      return img_to_array(image)
    else:
      return np.array([])
  except Exception as e:
    print (f"Error: {e}")
    return None

In [ ]:
image_list, label_list = [], []
try:
  print("[INFO] Loading images ...") 
  plant_disease_folder_list = listdir(train_dir)
  for plant_disease_folder in plant_disease_folder_list:
    print (f" [INFO] Processing {plant_disease_folder} ...") 
    plant_disease_image_list = listdir(f"{train_dir}/{plant_disease_folder}/")
    for image in plant_disease_image_list[:N_IMAGES]:
      image_directory = f"{train_dir}/{plant_disease_folder}/{image}" 
      if image_directory.endswith(".jpg")==True or image_directory.endswith(".JPG")==True: 
        image_list.append(convert_image_to_array(image_directory)) 
        label_list.append(plant_disease_folder)
  print("[INFO] Image loading completed") 
except Exception as e:
  print (f"Error: {e}")
# Transform the loaded training image data into numpy array 
np_image_list = np.array(image_list, dtype=np.float16) / 225.0 
print()
# Check the number of images loaded for training 
image_len = len(image_list) 
print(f"Total number of images: {image_len}")

[INFO] Loading images ...
 [INFO] Processing Tomato___Septoria_leaf_spot ...
 [INFO] Processing Tomato___Tomato_mosaic_virus ...
 [INFO] Processing Cherry_(including_sour)___Powdery_mildew ...
 [INFO] Processing Pepper,_bell___Bacterial_spot ...
 [INFO] Processing Tomato___Bacterial_spot ...
 [INFO] Processing Apple___Cedar_apple_rust ...
 [INFO] Processing Tomato___Tomato_Yellow_Leaf_Curl_Virus ...
 [INFO] Processing Strawberry___Leaf_scorch ...
 [INFO] Processing Tomato___Spider_mites Two-spotted_spider_mite ...
 [INFO] Processing Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot ...
 [INFO] Processing Soybean___healthy ...
 [INFO] Processing Squash___Powdery_mildew ...
 [INFO] Processing Potato___Late_blight ...
 [INFO] Processing Orange___Haunglongbing_(Citrus_greening) ...
 [INFO] Processing Peach___healthy ...
 [INFO] Processing Tomato___Early_blight ...
 [INFO] Processing Peach___Bacterial_spot ...
 [INFO] Processing Apple___Black_rot ...
 [INFO] Processing Grape___Black_rot ..

In [ ]:
label_binarizer = LabelBinarizer() 
image_labels=label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('plant_disease_label_transform.pkl', 'wb')) 
n_classes = len(label_binarizer.classes_)
print("Total number of classes: ", n_classes)

Total number of classes:  39


In [ ]:
print("[INFO] Splitting data to train and test...") 
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42)

[INFO] Splitting data to train and test...


In [ ]:
augment=ImageDataGenerator (rotation_range=25, width_shift_range=0.1, 
                            height_shift_range=0.1, shear_range=0.2, 
                            zoom_range=0.2, horizontal_flip=True, 
                            fill_mode="nearest")

In [ ]:
EPOCHS = 25
STEPS = 100
LR = 1e-3
BATCH_SIZE = 32
WIDTH = 256
HEIGHT = 256
DEPTH = 3

In [ ]:
import tensorflow as tf

In [ ]:
tf.keras.layers.BatchNormalization(
    axis=-1,
    momentum=0.99,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
)

In [ ]:
model=Sequential()
inputShape = (HEIGHT, WIDTH, DEPTH)
chanDim = -1
if K.image_data_format() =="channels_first": 
  inputShape (DEPTH, HEIGHT, WIDTH) 
  chanDim =1
model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape))
model.add(Activation ("relu"))
model.add(BatchNormalization(axis=chanDim)) 
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization (axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same")) 
model.add(Activation ("relu"))
model.add(BatchNormalization (axis=chanDim)) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization (axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu")) 
model.add(BatchNormalization (axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation ("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation ("softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 activation (Activation)     (None, 256, 256, 32)      0         
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 85, 85, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 85, 85, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 85, 85, 64)        1

In [ ]:
# Initialize optimizer
opt = Adam (lr=LR, decay=LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"]) 
print("[INFO] Training network...")
history = model.fit_generator (augment.flow(x_train, y_train, batch_size=BATCH                               validation_data=(x_test, y_test), 
                               steps_per_epoch=len(x_train) // BATCH_SIZE, 
                               epochs=EPOCHS,
                               verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[INFO] Training network...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


Epoch 1/25
97/97 [==============================] - 716s 7s/step - loss: 0.5078 - accuracy: 0.0716 - val_loss: 0.1560 - val_accuracy: 0.0346
Epoch 2/25
97/97 [==============================] - 716s 7s/step - loss: 0.1056 - accuracy: 0.2927 - val_loss: 1.3541 - val_accuracy: 0.0090
Epoch 3/25
97/97 [==============================] - 722s 7s/step - loss: 0.1350 - accuracy: 0.1762 - val_loss: 0.3295 - val_accuracy: 0.0756
Epoch 4/25
97/97 [==============================] - 723s 7s/step - loss: 0.1036 - accuracy: 0.3446 - val_loss: 0.1688 - val_accuracy: 0.1141
Epoch 5/25
97/97 [==============================] - 731s 8s/step - loss: 0.0856 - accuracy: 0.4725 - val_loss: 0.1669 - val_accuracy: 0.1962
Epoch 6/25
97/97 [==============================] - 733s 8s/step - loss: 0.0755 - accuracy: 0.5214 - val_loss: 0.0999 - val_accuracy: 0.3782
Epoch 7/25
97/97 [==============================] - 732s 8s/step - loss: 0.0736 - accuracy: 0.5415 - val_loss: 0.0964 - val_accuracy: 0.3769
Epoch 8/25
97

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy'] 
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
# Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy') 
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy') 
plt.title('Training and Validation accurarcy') 
plt.legend ()
plt.figure()
# Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
print("[INFO] Calculating model accuracy") 
scores = model.evaluate(x_test, y_test) 
print(f"Test Accuracy: {scores [1]*100}")

In [ ]:
def predict_disease (image_path):
  image_array = convert_image_to_array(image_path) 
  np_image=np.array(image_array, dtype=np. float16) / 225.0
  np_image=np.expand_dims (np_image, 0) 
  plt.imshow(plt.imread (image_path))
  result = model.predict_classes(np_image) 
  print((image_labels.classes_[result][0]))

In [ ]:
predict_disease('/content/Plantvillage/val/Blueberry_healthy/008c85d0-a954-4127-bd26-861dc8a1e6ff_ _RS_HL 2431.JPG')

In [ ]:
predict_disease('/content/Plantvillage/val/Potato Early blight/03b0d3c1-b5b0-48f4-98aa-f8904670290f_RS Early.B 7051.JPG')